<a href="https://colab.research.google.com/github/ArthurSehnem/cinematch/blob/main/cinematch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator

# Carregar dados
data = pd.read_csv('TMDB_movie_dataset_v11.csv', on_bad_lines='skip', nrows=10000)
general_data = data.copy()

# Selecionar e combinar os recursos
features = ['genres', 'overview', 'tagline', 'production_companies', 'keywords']
data_features = data[features].fillna(" ")
combined_features = (
    data_features['genres'] + ' ' +
    data_features['overview'] + ' ' +
    data_features['tagline'] + ' ' +
    data_features['production_companies'] + ' ' +
    data_features['keywords']
)

# Vetorização e similaridade
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
feature_vectors = vectorizer.fit_transform(combined_features)
similarity = cosine_similarity(feature_vectors)

# Entrada do usuário
movie_name = input('\n🎬 Digite o nome original (em inglês) de um filme que você gosta: ').strip().title()
list_movies = general_data['original_title'].tolist()
close_match = difflib.get_close_matches(movie_name, list_movies)

if not close_match:
    print("❌ Nenhum filme parecido encontrado.")
    exit()

matched_movie = close_match[0]
index_movie = general_data[general_data['original_title'] == matched_movie].index[0]

# Obter recomendações
similarity_score = list(enumerate(similarity[index_movie]))
sorted_similar_movie = sorted(similarity_score, key=lambda x: x[1], reverse=True)

# Mostrar top 5, ignorando o próprio filme
print(f"\n🔎 Recomendando com base no filme: {matched_movie}")
print('\n🎯 Filmes recomendados para você:\n')

recommendations_shown = 0
shown_titles_and_summaries = set()

for i in sorted_similar_movie:
    index = i[0]
    if index == index_movie:
        continue

    title = general_data.iloc[index]['original_title']
    overview = general_data.iloc[index]['overview']

    identifier = (title.strip().lower(), overview.strip().lower())
    if identifier in shown_titles_and_summaries:
        continue
    shown_titles_and_summaries.add(identifier)

    score = general_data.iloc[index]['vote_average']
    release_date = general_data.iloc[index]['release_date']

    try:
        overview_pt = GoogleTranslator(source='auto', target='pt').translate(overview)
    except:
        overview_pt = "⚠️ Erro ao traduzir sinopse."

    try:
        formatted_date = pd.to_datetime(release_date, errors='coerce').strftime('%d/%m/%Y')
    except:
        formatted_date = "Data desconhecida"

    print(f"{recommendations_shown + 1}. {title} ({formatted_date}) - Nota: {score:.1f}" if not pd.isna(score) else f"{recommendations_shown + 1}. {title} ({formatted_date}) - Nota: N/A")
    print(f"   Sinopse: {overview_pt}\n")

    recommendations_shown += 1
    if recommendations_shown >= 5:
        break



🎬 Digite o nome original (em inglês) de um filme que você gosta: Finding Nemo

🔎 Recomendando com base no filme: Finding Nemo

🎯 Filmes recomendados para você:

1. The Reef (15/05/2010) - Nota: 5.7
   Sinopse: Um grande tubarão branco caça a tripulação de um veleiro capsado ao longo da grande barreira recife.

2. Shark Tale (20/09/2004) - Nota: 6.0
   Sinopse: Oscar é um peixe pequeno cujas grandes aspirações geralmente o causam problemas. Enquanto isso, Lenny é um ótimo tubarão branco, com um segredo surpreendente de que nenhuma criatura marinha adivinharia: ele é vegetariano. Quando uma mentira transforma Oscar em um herói improvável e Lenny se torna um pária, os dois formam uma amizade improvável.

3. Finding Dory (16/06/2016) - Nota: 7.0
   Sinopse: Dory se reúne com seus amigos Nemo e Marlin na busca por respostas sobre seu passado. O que ela pode se lembrar? Quem são seus pais? E onde ela aprendeu a falar baleia?

4. Great White (13/05/2021) - Nota: 5.3
   Sinopse: Um avião marí